In [1]:
from tqdm.notebook import tqdm as progress
import pandas as pd
import plotly.express as px
from plotly.offline import plot
import dimcli
from dimcli.utils import *
import os, sys, time, json
from IPython.display import Image
from IPython.core.display import HTML 
import csv
import ast

/Users/FionaXu/Documents/Research/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning:

urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020



In [2]:
#Log into Dimensions
#API key is: 9F8D648F0D7E437CB1736BEBDF007F02
#!pip install dimcli -U --quiet 

print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
    import getpass
    KEY = getpass.getpass(prompt='API Key: ')  
    dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
    KEY = "9F8D648F0D7E437CB1736BEBDF007F02"
    dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()



==
Logging in..
Dimcli - Dimensions API Client (v1.4)
Connected to: <https://app.dimensions.ai/api/dsl> - DSL v2.10
Method: manual login


In [3]:

#Link to grant variables description: https://docs.dimensions.ai/dsl/datasource-grants.html#grants
search_Var = "[abstract + active_year + category_bra + category_for + category_for_2020 + category_hra + category_hrcs_hc + category_hrcs_rac + category_icrp_cso + category_icrp_ct + category_rcdc + category_sdg + category_uoa + concepts + concepts_scores + date_inserted + dimensions_url + end_date + foa_number + funder_countries + funder_org_acronym + funder_org_cities + funder_org_countries + funder_org_name + funder_org_states + funder_orgs + funders + funding_aud + funding_cad + funding_chf + funding_cny + funding_currency + funding_eur + funding_gbp + funding_jpy + funding_nzd + funding_org_acronym + funding_org_cities + funding_org_city + funding_org_name + funding_org_states + funding_schemes + funding_usd + grant_number + id + investigators + keywords + language + language_title + linkout + original_title + project_numbers + research_org_cities + research_org_countries + research_org_names + research_org_state_codes + research_orgs + researchers + score + start_date + start_year + title]"
names = {}

#CHANGE CSV FILE TO THE NAME OF YOUR FILE SAVED FROM DimensionsAI_IDs.ipynb
with open('150physicianscientists.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)  
    for row in reader:
        name = row[0]
        openIds_str = row[1]  
        openIds_list = ast.literal_eval(openIds_str)
        names[name] = openIds_list

print(names)


FileNotFoundError: [Errno 2] No such file or directory: '1995_2010.csv'

In [13]:
def findby_idGrant(nameDict):
    all_Grants= {}
    count=0
    impactCodes = ['R01', 'R03', 'R13', 'R15', 'R21', 'R34', 'R41', 'R42', 'R43', 'R44', 'R56', 'R29', 'F05', 'F30', 'F31', 'F32', 'F33', 'F37', 'F38', 'F99', 'F12', 'FM1', 'R2F', 'RF1']
    for name, person_ids in nameDict.items():
        indv_grant_list = []
        for i in person_ids:
            numGrants = 0
            res = dsl.query_iterative(f"""search grants where researchers = "{i}" return grants{search_Var}""")
            for grant in res.json['grants']: #Filtering out grants with 5 or more investigators and those that are NIH but don't have impactful codes
                caught = [code for code in impactCodes if(code in grant['grant_number'])] #bool(caught) returns false for empty list (if grant has no impactful code)
                if(len(grant['investigators']) < 5 and (bool(caught) or not "National" in grant['funder_org_name'])):
                    indv_grant_list.append(grant) 
                    numGrants+=1
            number = res.json['_stats']['total_count']
            print(f'{i} id has {number} total grants')
            print(f'{i} id has {numGrants} high impact grants')
            listDict = res.json.get('grants', [])

            indv_grant_list.extend(listDict)

        all_Grants[name] = indv_grant_list
        count+=1
        print(f'{count}. {name}')
        print(len(all_Grants[name]))
    return all_Grants

grants = findby_idGrant(names) 

json_filename = "all_Grants.json"

with open(json_filename, 'w') as json_file:
    json.dump(grants, json_file, indent=2)

print(f'The data has been written to {json_filename}')

Starting iteration with limit=1000 skip=0 ...
===
Records extracted: 0
Warnings:  8
Starting iteration with limit=1000 skip=0 ...


ur.0627200551.43 id has 0 total grants
ur.0627200551.43 id has 0 high impact grants


0-5 / 5 (0.71s)
===
Records extracted: 5
Warnings:  8
Starting iteration with limit=1000 skip=0 ...


ur.01271177722.98 id has 5 total grants
ur.01271177722.98 id has 5 high impact grants
1. Alan Hillman
10


0-9 / 9 (0.40s)
===
Records extracted: 9
Warnings:  8
Starting iteration with limit=1000 skip=0 ...


ur.0651722340.16 id has 9 total grants
ur.0651722340.16 id has 4 high impact grants
2. Mary Dinauer
13


KeyboardInterrupt: 

In [2]:
import csv
import json

csv_filename = "dimensionsGrants_data.csv"

with open("all_Grants.json", 'r', encoding='utf-8') as file:
    personDict = json.load(file)
# removed 'investigators
common_work_info_headers = ['abstract', 'active_years', 'altmetric', 'associated_grant_ids', 'brief_title', 'category_bra', 'category_for', 'category_for_2020', 'category_hra', 'category_hrcs_rac', 'category_rcdc', 'conditions', 'date', 'date_inserted', 'dimensions_url', 'end_date', 'funder_countries', 'funders', 'funding_usd','grant_number','gender', 'id', 'interventions', 'linkout', 'mesh_terms', 'phase', 'publication_ids', 'publications', 'registry', 'score', 'start_date', 'study_arms', 'study_designs', 'study_eligibility_criteria', 'study_maximum_age', 'study_minimum_age', 'study_outcome_measures', 'study_participants', 'study_type', 'title']


with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    
    header = ['Person'] + common_work_info_headers
    writer.writerow(header)

    # Write data rows
    for person, works in personDict.items():
        for work in works:  # Assuming works is a list of dictionaries now
            row = [person] + [work.get(head, '') for head in common_work_info_headers]
            writer.writerow(row)

print(f"CSV file '{csv_filename}' created successfully.")

CSV file 'dimensionsGrants_data.csv' created successfully.
